In [1]:
import pymongo
import pandas as pd

data_mongo= pymongo.MongoClient('localhost:27017')

db= data_mongo['test']

In [2]:
def mongo_data(collection):
    mongoColle= db[collection]
    data= list(mongoColle.find())
    df= pd.json_normalize(data,sep='_')
    df._id=df._id.astype(str)
    return df

In [3]:
from bson import ObjectId
def setle_clean(select):
    de= db['settlements']
    obj= de.find_one({'_id':ObjectId(select)})
    df_setle= pd.json_normalize(obj,sep='_')
    df_setle['id_central']=df_setle.centralPoint.apply(lambda x: x[0]['_id'] if '_id' in x[0] else None)
    df_setle['latitud_c']=df_setle.centralPoint.apply(lambda x: x[0]['lat'] if 'lat' in x[0] else None)
    df_setle['longitud_c']=df_setle.centralPoint.apply(lambda x: x[0]['lng'] if 'lng' in x[0] else None)
    setle_n = df_setle[['_id','hectares','registerNumber','headsCount','name','latitud_c','longitud_c']]
    return setle_n

def selec_setle(data,select):
    return data[data._id==select]


In [4]:
d= setle_clean('63ecf27ba9f1a40025792acf')
d

,_id,hectares,registerNumber,headsCount,name,latitud_c,longitud_c
0,63ecf27ba9f1a40025792acf,1.09,None,None,MACSA,-34.164999,-64.07001


In [5]:
df_animal=mongo_data('animals')
df_devis= mongo_data('devices')
df_plot= mongo_data('plots')
df_setith= mongo_data('settlementithcounts')
df_setle= mongo_data('settlements')

In [6]:
df_devis.deviceType.unique()

array(['CARAVANA', 'COLLAR', 'PUNTO FIJO'], dtype=object)

In [7]:
df_plot['latitud']=df_plot.geoPoints.apply(lambda x: x[0]['lat'])
df_plot['longitud']=df_plot.geoPoints.apply(lambda x: x[0]['lng'])
df_plot['id_geoPoint']=df_plot.geoPoints.apply(lambda x: x[0]['_id'] if '_id' in x[0] else None)

In [8]:
df_setle['id_limits']=df_setle.limits.apply(lambda x: x[0]['_id'] if len(x)>0 else None )
df_setle['latitud']=df_setle.limits.apply(lambda x: x[0]['lat'] if len(x)>0 else None )
df_setle['longitud']=df_setle.limits.apply(lambda x: x[0]['lng'] if len(x)>0 else None )

In [9]:
df_setle['id_central']=df_setle.centralPoint.apply(lambda x: x[0]['_id'] if '_id' in x[0] else None)
df_setle['latitud_c']=df_setle.centralPoint.apply(lambda x: x[0]['lat'] if 'lat' in x[0] else None)
df_setle['longitud_c']=df_setle.centralPoint.apply(lambda x: x[0]['lng'] if 'lng' in x[0] else None)

In [10]:
df_animal['animalSettlement']=df_animal['animalSettlement'].apply(lambda x:x[0])
df_animal.animalSettlement=df_animal.animalSettlement.astype(str)

In [11]:
def conect_animal():
    df_animal=mongo_data('animals')
    df_animal['animalSettlement']=df_animal['animalSettlement'].apply(lambda x:x[0])
    df_animal.animalSettlement=df_animal.animalSettlement.astype(str)
    result= df_animal[(df_animal.caravanaNumber.str.contains('AGUADA'))|(df_animal.caravanaNumber.str.contains('PUNTO_FIJO'))]#lo use para extraer un csv con aguadas y puntos fijos
    return result

In [12]:
df_animal[df_animal.animalSettlement=='63ecf27ba9f1a40025792acf']

,_id,animalSettlement,caravanaNumber,animalType,weight,birthDay,breed,isPregmant,dateOfBirth,observations,createdAt,updatedAt,__v,animalTags,animalSenasa
90,63ed0ae9a9f1a40025793c1d,63ecf27ba9f1a40025792acf,B10119,SIN ESPECIFICAR,NaN,NaT,SIN ESPECIFICAR,NaN,NaT,NaN,2023-02-15 16:40:09.824,2023-02-15 16:40:09.824,0,[],JO278-Q550-0
91,63ed0b15a9f1a40025793c52,63ecf27ba9f1a40025792acf,B10124,SIN ESPECIFICAR,NaN,NaT,SIN ESPECIFICAR,NaN,NaT,NaN,2023-02-15 16:40:53.331,2023-02-15 16:40:53.331,0,[],CCO14-I054-1
92,63ed0b7aa9f1a40025793c82,63ecf27ba9f1a40025792acf,B10194,SIN ESPECIFICAR,NaN,NaT,SIN ESPECIFICAR,NaN,NaT,NaN,2023-02-15 16:42:34.667,2023-02-15 16:42:34.667,0,[],JF718-B858-8
93,63ed0b8da9f1a40025793ce6,63ecf27ba9f1a40025792acf,B10125,SIN ESPECIFICAR,NaN,NaT,SIN ESPECIFICAR,NaN,NaT,NaN,2023-02-15 16:42:53.464,2023-02-15 16:42:53.464,0,[],JO278-P809-1
94,63ed0b93a9f1a40025793d0f,63ecf27ba9f1a40025792acf,B10139,SIN ESPECIFICAR,NaN,NaT,SIN ESPECIFICAR,NaN,NaT,NaN,2023-02-15 16:42:59.098,2023-02-15 16:42:59.098,0,[],JO278-Q551-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,64235c730dac550030135fb2,63ecf27ba9f1a40025792acf,TBOZAVI8,VACA,100.0,2022-06-18,HEREFORD,False,NaT,,2023-03-28 21:30:27.676,2023-03-28 21:30:27.676,0,[],TESTBOZAVI8
624,64235c730dac550030135fb4,63ecf27ba9f1a40025792acf,TBOZAVI7,VACA,100.0,2022-06-18,HEREFORD,False,NaT,,2023-03-28 21:30:27.683,2023-03-28 21:30:27.683,0,[],TESTBOZAVI7
625,642367fd0dac550030136043,63ecf27ba9f1a40025792acf,TBOZAVI9,VACA,100.0,2022-06-18,HEREFORD,False,NaT,,2023-03-28 22:19:41.682,2023-03-28 22:19:41.682,0,[],TESTBOZAVI9
626,642367fd0dac550030136047,63ecf27ba9f1a40025792acf,TBOZAVI11,VACA,100.0,2022-06-18,HEREFORD,False,NaT,,2023-03-28 22:19:41.690,2023-03-28 22:19:41.690,0,[],TESTBOZAVI11


In [31]:
df_setle

,_id,enabledUsers,plots,limits,hectares,registerNumber,headsCount,name,createdAt,updatedAt,__v,centralPoint,auravantApiKey,enabled,id_limits,latitud,longitud,id_central,latitud_c,longitud_c
0,620e6e5e60543d0026a01f0e,"[6201245070900d002592d7ec, 636c17ea2bd5b200305...","[63b55f4b973f9f35c0c5b37d, 63b55f4b973f9f35c0c...","[{'_id': 63cedb56ea0de8476c39744d, 'lat': -32....",5000.00,XCSS12345,10.0,La Florida,2022-02-17 15:48:47.058,2023-02-17 15:11:39.739,0,"[{'_id': 63cedb56ea0de8476c397457, 'lat': -32....",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzcmMiO...,True,63cedb56ea0de8476c39744d,-32.852009,-63.641726,63cedb56ea0de8476c397457,-32.852351,-63.639977
1,63877cd7e1ed590030662d0a,[6201245070900d002592d7ec],"[63877cd7e1ed590030662d11, 63d116f852372700318...","[{'_id': 63cedb6dea0de8476c39747b, 'lat': -33....",100.00,12311413,100.0,Stock Test,2022-11-30 15:55:03.627,2023-02-07 21:47:03.299,0,"[{'_id': 63cedb6dea0de8476c39747f, 'lat': -33....",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzcmMiO...,False,63cedb6dea0de8476c39747b,-33.128746,-64.349935,63cedb6dea0de8476c39747f,-33.129187,-64.349629
2,63e641fc22ee080030ba838b,[6201245070900d002592d7ec],[63f8c5c6110f66003023ec90],"[{'_id': 63f01465e572c6002f4d73f2, 'lat': -32....",0.21,None,NaN,prueba,2023-02-10 13:09:16.013,2023-02-27 19:07:42.186,0,"[{'_id': 63f01465e572c6002f4d73f5, 'lat': -32....",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzcmMiO...,False,63f01465e572c6002f4d73f2,-32.982682,-68.775564,63f01465e572c6002f4d73f5,-32.982998,-68.775196
3,63e6454922ee080030ba8728,[6201245070900d002592d7ec],"[63e6456b22ee080030ba8756, 6414cf9bb2505016885...","[{'_id': 63e6454922ee080030ba8729, 'lat': -32....",0.57,None,NaN,establecimiento de prueba,2023-02-10 13:23:21.915,2023-04-03 18:39:57.746,0,"[{'_id': 63e6454922ee080030ba872d, 'lat': -32....",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzcmMiO...,False,63e6454922ee080030ba8729,-32.996429,-60.634345,63e6454922ee080030ba872d,-32.996823,-60.633884
4,63efd78d65980f4378711b6f,[6201245070900d002592d7ec],[63f8fd2a3c8b18002f20a41e],"[{'_id': 63efd78d65980f4378711b70, 'lat': -32....",0.02,None,NaN,prueba macsa,2023-02-17 19:37:49.367,2023-04-03 18:39:48.051,0,"[{'_id': 63ecf27ba9f1a40025792ad4, 'lat': -34....",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzcmMiO...,False,63efd78d65980f4378711b70,-32.996636,-60.634266,63ecf27ba9f1a40025792ad4,-34.164999,-64.070010
5,63ecf27ba9f1a40025792acf,"[6201245070900d002592d7ec, 63ecb2e2a9f1a400257...","[63eff7fbebcff60025f5d968, 63effaa9eb90f92734f...","[{'_id': 63ecf27ba9f1a40025792ad0, 'lat': -34....",1.09,None,NaN,MACSA,2023-02-15 14:55:55.972,2023-03-23 14:29:30.396,0,"[{'_id': 63ecf27ba9f1a40025792ad4, 'lat': -34....",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzcmMiO...,True,63ecf27ba9f1a40025792ad0,-34.163851,-64.071678,63ecf27ba9f1a40025792ad4,-34.164999,-64.070010


In [13]:
def update_aguada(setle):
    df_devis= mongo_data('devices')
    df_devis.deviceAnimalID=df_devis.deviceAnimalID.astype(str)
    data_devise = df_devis[df_devis.deviceType=='PUNTO FIJO'] 
    aguadas= conect_animal()
    x= aguadas[aguadas['animalSettlement']==setle]
    agua =data_devise[data_devise.deviceAnimalID.isin(x._id)]
    return agua


In [14]:
prueba= df_devis[df_devis.deviceMACAddress.isin(['0004A30B00F860CC', '0004A30B00F82E81', '0004A30B00F89C5D',
       '0004A30B00F8302C', '0004A30B00F869B4', '0004A30B00F86440',
       '0004A30B00F8A181', '0004A30B00F841E4', '0004A30B00F8B356',
       '0004A30B00F81069', '0004A30B00F842C1', '0004A30B00F8B217',
       '0004A30B00F81075', '0004A30B00F83085', '0004A30B00F842B6',
       '0004A30B00F89C52', '0004A30B00F862AF', '0004A30B00F7E0C6',
       '0004A30B00F82EB9'])]    

In [15]:
df_animal[df_animal._id.isin(prueba.deviceAnimalID)]

,_id,animalSettlement,caravanaNumber,animalType,weight,birthDay,breed,isPregmant,dateOfBirth,observations,createdAt,updatedAt,__v,animalTags,animalSenasa


In [16]:
masca =~df_setle._id.isin(df_animal.animalSettlement.unique())


In [32]:
df_animal[df_animal.animalSettlement=='620e6e5e60543d0026a01f0e']

,_id,animalSettlement,caravanaNumber,animalType,weight,birthDay,breed,isPregmant,dateOfBirth,observations,createdAt,updatedAt,__v,animalTags,animalSenasa
0,62d6e9920408c608acb6fdd0,620e6e5e60543d0026a01f0e,CAR9,VACA,500.0,2022-07-18 00:00:00.000,SIMBRAH,False,NaT,,2022-07-19 17:27:46.820,2023-02-10 18:40:12.193,0,[VIEJA],NaN
1,62d6e9d30408c608acb6fe08,620e6e5e60543d0026a01f0e,CAR71,NOVILLO,234.0,2022-07-18 00:00:00.000,SIMBRAH,False,NaT,,2022-07-19 17:28:51.345,2022-11-25 02:11:05.689,0,[PARICION],NaN
2,62d98db9a30a66002645c287,620e6e5e60543d0026a01f0e,CAR52,VACA,155.0,2022-07-18 00:00:00.000,BRANGUS,False,NaT,,2022-07-21 17:32:41.363,2022-07-21 17:32:41.363,0,NaN,NaN
3,62d99045a30a66002645c384,620e6e5e60543d0026a01f0e,CAR169,TERNERO,73.0,2022-07-18 00:00:00.000,INDUSIN,False,NaT,,2022-07-21 17:43:33.443,2022-07-21 17:43:33.443,0,NaN,NaN
4,62d9a88c0223ce002612afed,620e6e5e60543d0026a01f0e,CAR19,TERNERO,45.0,2022-03-25 00:00:00.000,BRAFORD,False,NaT,,2022-07-21 19:27:08.791,2022-12-06 14:08:45.687,0,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,640b79fde5142a002f656029,620e6e5e60543d0026a01f0e,PUNTO_FIJO_10,VACA,0.0,2023-03-10 18:42:05.749,DEFAULT-PUNTO-FIJO,False,NaT,NaN,2023-03-10 18:42:05.749,2023-03-10 18:42:05.749,0,[],NaN
211,640b7a20e5142a002f65603a,620e6e5e60543d0026a01f0e,PUNTO_FIJO_11,VACA,0.0,2023-03-10 18:42:41.026,DEFAULT-PUNTO-FIJO,False,NaT,NaN,2023-03-10 18:42:41.026,2023-03-10 18:42:41.026,0,[],NaN
212,640b7a4be5142a002f65604e,620e6e5e60543d0026a01f0e,PUNTO_FIJO_12,VACA,0.0,2023-03-10 18:43:24.140,DEFAULT-PUNTO-FIJO,False,NaT,NaN,2023-03-10 18:43:24.140,2023-03-10 18:43:24.140,0,[],NaN
213,640b7a61e5142a002f65605f,620e6e5e60543d0026a01f0e,PUNTO_FIJO_13,VACA,0.0,2023-03-10 18:43:46.226,DEFAULT-PUNTO-FIJO,False,NaT,NaN,2023-03-10 18:43:46.226,2023-03-10 18:43:46.226,0,[],NaN


In [17]:
df_setle[masca]#estos valores no tienen regitras con animales

,_id,enabledUsers,plots,limits,hectares,registerNumber,headsCount,name,createdAt,updatedAt,__v,centralPoint,auravantApiKey,enabled,id_limits,latitud,longitud,id_central,latitud_c,longitud_c
6,63ff75624c2d6d003084c117,[6201245070900d002592d7ec],[],"[{'_id': 63ff75624c2d6d003084c118, 'lat': -26....",61.5600,NaN,NaN,Prueba,2023-03-01 15:55:14.078,2023-03-01 15:55:45.760,0,"[{'_id': 63ff75624c2d6d003084c11c, 'lat': -26....",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzcmMiO...,False,63ff75624c2d6d003084c118,-26.853581,-65.252918,63ff75624c2d6d003084c11c,-26.858855,-65.251419
7,642b1d27cc00091984864f0a,[6201245070900d002592d7ec],[642b1d27cc00091984864f0d],[],79.9866,,0.0,prueba macsa,2023-04-03 18:38:31.434,2023-04-03 18:39:36.413,0,[{'_id': 642b1d27cc00091984864f0b}],NaN,False,None,NaN,NaN,642b1d27cc00091984864f0b,NaN,NaN
8,642c0b596490e600305e1819,[6201245070900d002592d7ec],[642c0e4a6490e600305e18eb],"[{'_id': 642c0b596490e600305e181a, 'lat': -26....",11.3300,None,NaN,Test,2023-04-04 11:34:49.097,2023-04-04 11:47:23.175,0,"[{'_id': 642c0b596490e600305e181e, 'lat': -26....",NaN,True,642c0b596490e600305e181a,-26.857838,-65.247076,642c0b596490e600305e181e,-26.859657,-65.245558


In [35]:
df_devis.deviceAnimalID= df_devis.deviceAnimalID.astype(str)
df_devis[df_devis.deviceAnimalID=='62d98db9a30a66002645c287']

,_id,deviceType,deviceFirmwareVersion,deviceMACAddress,deviceAnimalID,__v
20,62d98df8a30a66002645c2a8,CARAVANA,2222,A7AA2D0FF0F3,62d98db9a30a66002645c287,0


In [18]:
mascara= df_setle._id.isin(['63ff75624c2d6d003084c117','642b1d27cc00091984864f0a','642c0b596490e600305e1819'])
df_setle.drop(df_setle[mascara].index,inplace=True)

In [19]:
df_setle.to_csv('settle.csv',index=False)

In [20]:
df_plot[df_plot._id=='642c0e4a6490e600305e18eb']

,_id,auravant,virtualFenceGeoPoints,name,geoPoints,batchNumber,forage,specie,antiquity,quality,...,hectares,color,lat,lng,auravant_name,auravant_auravantPlotId,auravant_auravantSettlId,latitud,longitud,id_geoPoint
79,642c0e4a6490e600305e18eb,NaN,[],P1,"[{'_id': 642c0e4a6490e600305e18ec, 'lat': -26....",123123,LEGUMINOSAS,Alfalfa sin reposo invernal,None,EXCELENTE,...,1.08,#8db42f,NaN,NaN,NaN,NaN,NaN,-26.859093,-65.246753,642c0e4a6490e600305e18ec


In [21]:
df_setle[df_setle._id=='63ecf27ba9f1a40025792acf']

,_id,enabledUsers,plots,limits,hectares,registerNumber,headsCount,name,createdAt,updatedAt,__v,centralPoint,auravantApiKey,enabled,id_limits,latitud,longitud,id_central,latitud_c,longitud_c
5,63ecf27ba9f1a40025792acf,"[6201245070900d002592d7ec, 63ecb2e2a9f1a400257...","[63eff7fbebcff60025f5d968, 63effaa9eb90f92734f...","[{'_id': 63ecf27ba9f1a40025792ad0, 'lat': -34....",1.09,None,NaN,MACSA,2023-02-15 14:55:55.972,2023-03-23 14:29:30.396,0,"[{'_id': 63ecf27ba9f1a40025792ad4, 'lat': -34....",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzcmMiO...,True,63ecf27ba9f1a40025792ad0,-34.163851,-64.071678,63ecf27ba9f1a40025792ad4,-34.164999,-64.07001


In [22]:
# agrego una columna para identificar a el settlements 
for i,d in df_setle.iterrows():
    id_plot=[str(x).strip() for x in d[['plots']].values[0]]
    if len(id_plot)!=0:
        df_plot.loc[df_plot._id.isin(id_plot),'id_settlements'] = df_setle.loc[i][['_id']].values[0]


In [23]:
df_setith.settlementId =df_setith.settlementId.astype(str)

In [24]:
pru= df_setith[df_setith.settlementId =='63ecf27ba9f1a40025792acf']
pru

,_id,settlementId,ITH,createdAt,updatedAt,__v
13265,63f77f71d3575e0030341d6d,63ecf27ba9f1a40025792acf,77.0,2023-02-23 15:00:01.487,2023-02-23 15:00:01.487,0
13275,63f78d8286f2a4002f3f679f,63ecf27ba9f1a40025792acf,78.0,2023-02-23 16:00:02.344,2023-02-23 16:00:02.344,0
13279,63f79b9186f2a4002f3f681c,63ecf27ba9f1a40025792acf,79.0,2023-02-23 17:00:01.566,2023-02-23 17:00:01.566,0
13285,63f7a9a186f2a4002f3f689f,63ecf27ba9f1a40025792acf,79.0,2023-02-23 18:00:01.770,2023-02-23 18:00:01.770,0
13295,63f7b7b186f2a4002f3f6a32,63ecf27ba9f1a40025792acf,79.0,2023-02-23 19:00:01.914,2023-02-23 19:00:01.914,0
...,...,...,...,...,...,...
17765,642d62c1b239c60030f955e5,63ecf27ba9f1a40025792acf,64.0,2023-04-05 12:00:01.949,2023-04-05 12:00:01.949,0
17768,642d70d3b239c60030f9586f,63ecf27ba9f1a40025792acf,67.0,2023-04-05 13:00:03.246,2023-04-05 13:00:03.246,0
17771,642d7ee1b239c60030f95a50,63ecf27ba9f1a40025792acf,69.0,2023-04-05 14:00:01.594,2023-04-05 14:00:01.594,0
17775,642d8cf1b239c60030f95c51,63ecf27ba9f1a40025792acf,71.0,2023-04-05 15:00:01.878,2023-04-05 15:00:01.878,0


In [25]:
def time_min_max_data(data):
    """Funcion que devuelve los valores de fecha maxima que aparece en los dataframe

    Args:
        data (dataframe): recibe el dataframe saca fecha maxima y fecha minima

    Returns:
        tuple:(datatime,datatime) 
    """
    fecha_max=data.createdAt.dt.date.max().strftime('%Y-%m-%d')
    fecha_min=data.createdAt.dt.date.min().strftime('%Y-%m-%d')
    return fecha_min ,fecha_max

In [26]:
time_min_max_data(pru)

('2023-02-23', '2023-04-05')

In [27]:
prueba=pru[pru.createdAt.dt.date ==pd.to_datetime('2023-02-24')]
prueba

C:\Users\Usuario\AppData\Local\Temp\ipykernel_10720\442622768.py:1: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  prueba=pru[pru.createdAt.dt.date ==pd.to_datetime('2023-02-24')]


,_id,settlementId,ITH,createdAt,updatedAt,__v
13327,63f7fe01d9d7de00304a2ff8,63ecf27ba9f1a40025792acf,72.0,2023-02-24 00:00:01.158,2023-02-24 00:00:01.158,0
13338,63f7fe01ffec060030bfb504,63ecf27ba9f1a40025792acf,72.0,2023-02-24 00:00:01.259,2023-02-24 00:00:01.259,0
13346,63f80c116e68a70030edbc01,63ecf27ba9f1a40025792acf,72.0,2023-02-24 01:00:01.808,2023-02-24 01:00:01.808,0
13354,63f81a226e68a70030edc685,63ecf27ba9f1a40025792acf,70.0,2023-02-24 02:00:02.589,2023-02-24 02:00:02.589,0
13358,63f828316e68a70030edd0ef,63ecf27ba9f1a40025792acf,69.0,2023-02-24 03:00:01.487,2023-02-24 03:00:01.487,0
13368,63f836436e68a70030edd13c,63ecf27ba9f1a40025792acf,68.0,2023-02-24 04:00:03.798,2023-02-24 04:00:03.798,0
13369,63f844516e68a70030edd1ca,63ecf27ba9f1a40025792acf,67.0,2023-02-24 05:00:01.625,2023-02-24 05:00:01.625,0
13382,63f852636e68a70030edd1f2,63ecf27ba9f1a40025792acf,66.0,2023-02-24 06:00:03.937,2023-02-24 06:00:03.937,0
13387,63f860716e68a70030edd21b,63ecf27ba9f1a40025792acf,65.0,2023-02-24 07:00:01.842,2023-02-24 07:00:01.842,0
13396,63f86e826e68a70030edd278,63ecf27ba9f1a40025792acf,64.0,2023-02-24 08:00:02.621,2023-02-24 08:00:02.621,0


In [28]:
prueba.ITH.mean()

71.88